In [7]:
!python --version

Python 3.8.3


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import log_loss, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import time
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import wait, ALL_COMPLETED, as_completed
from tqdm import tqdm
import joblib
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
data_dir = './DATA/lish-moa/'

***
# Table of Contents   
1. Load the dataset  
2. Preprocessing   
3. Models : Binary Relevance (Random Forest)  
***

## Load the dataset

In [3]:
X = pd.read_csv(data_dir+'train_features.csv', index_col='sig_id')
y = pd.read_csv(data_dir+'train_targets_scored.csv', index_col='sig_id')

## Preprocessing

In [4]:
# One-hot encoding for cp_type and cp_dose
X['cp_type'].replace({'trt_cp':1., 'ctl_vehicle':0.}, inplace=True)
X['cp_dose'].replace({'D1':1., 'D2':0.}, inplace=True)

# split into training set and test set
SEED = 123
np.random.seed(SEED)

ids = X.index.values.copy()
np.random.shuffle(ids)

train_perc, test_perc = 0.85, 0.15
train_id = ids[:round(len(ids)*train_perc)]
test_id = ids[round(len(ids)*train_perc):]

X_train = X.loc[train_id]
X_test = X.loc[test_id]

y_train = y.loc[train_id]
y_test = y.loc[test_id]

# normalize the data
scaler = StandardScaler()
X_train_norm = pd.DataFrame(scaler.fit_transform(X_train))
X_train_norm.columns = X_train.columns
X_train_norm.index = X_train.index

X_test_norm = pd.DataFrame(scaler.transform(X_test))
X_test_norm.columns = X_test.columns
X_test_norm.index = X_test.index

## Model

In [12]:
# 
pca = PCA(n_components=500)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_norm))
X_train_pca.index = X_train.index

X_test_pca = pd.DataFrame(pca.transform(X_test_norm))
X_test_pca.index = X_test.index

In [14]:
weight_ratio = 0.73
n_estimators = 120
max_depth = 4
min_samples_leaf = 1

model_dic_rf = {}
X = X_train_pca.values
for col_name, y in tqdm(y_train.iteritems()):
    # class_weight for each target column
    class_weight = {0:1, 1:min(round(len(y)/sum(y)-1)*weight_ratio, 8000)} 
    
#     pca = PCA(n_components = n_components)
    rfc = RandomForestClassifier( n_estimators = n_estimators,
                                  max_depth = max_depth,
                                  class_weight = class_weight,
                                  min_samples_leaf = min_samples_leaf,
                                  n_jobs = -1,
                                  random_state=100);
#     pipe = Pipeline(steps=[('pca',pca), ('rfc', rfc)])
    rfc.fit(X, y);
    model_dic_rf[col_name] = rfc

# inference
y_train_predict = pd.DataFrame([], columns=y_train.columns)
y_test_predict = pd.DataFrame([], columns=y_train.columns)
X = X_train_pca.values
X2 = X_test_pca.values
for i, rfc in tqdm(model_dic_rf.items()):
    y_train_predict[i] = rfc.predict(X)
    y_test_predict[i] = rfc.predict(X2)

206it [25:26,  7.41s/it]
100%|██████████| 206/206 [23:57<00:00,  6.98s/it]


In [18]:
# binary relevance
# print('n_components:', n_components)
print('weight_ratio: ', weight_ratio,' n_estimators:',n_estimators,' max_depth: ', max_depth, ' max_features:',max_features, ' min_samples_leaf:',min_samples_leaf)
print('Precision: ')
print(precision_score(y_train.values.reshape(-1, 1), y_train_predict.values.reshape(-1, 1)))
print('Recall: ')
print(recall_score(y_train.values.reshape(-1, 1), y_train_predict.values.reshape(-1, 1)))
print('F1: ')
print(f1_score(y_train.values.reshape(-1, 1), y_train_predict.values.reshape(-1, 1)))
print()
print('Precision: ')
print(precision_score(y_test.values.reshape(-1, 1), y_test_predict.values.reshape(-1, 1)))
print('Recall: ')
print(recall_score(y_test.values.reshape(-1, 1), y_test_predict.values.reshape(-1, 1)))
print('F1: ')
print(f1_score(y_test.values.reshape(-1, 1), y_test_predict.values.reshape(-1, 1)))
print()
print(y_train.sum().sum())
print(y_train_predict.sum().sum())
print()
print(y_test.sum().sum())
print(y_test_predict.sum().sum())

weight_ratio:  0.73  n_estimators: 120  max_depth:  4  max_features: 30  min_samples_leaf: 1
Precision: 
0.8950437317784257
Recall: 
0.5576358809557077
F1: 
0.6871556473829201

Precision: 
0.7209302325581395
Recall: 
0.19604743083003953
F1: 
0.30826600372902424

14314
8918

2530
688


### Grid search with cross-validatation

In [61]:
folds = 5

current = 0
start = 0
best_rfc = {}
best_params = {}
X = X_train.values
def search_model(i, col_name, y):
    global best_rfc
    global best_params
    global current
    global start
    
    start += 1
    start_time = time.process_time()
    print("Start " + str(start) + " ")

    ratio = round(len(y)/sum(y)-1)
    tuned_params = {
        'class_weight':[{0:1, 1:round(weight_ratio*ratio)} for weight_ratio in [0.71, 0.72, 0.73, 0.74, 0.75, 0.76]],
        'max_depth': [3,4,5,6,7],
        'max_features':[25, 30, 35]
    }
    rfc = RandomForestClassifier(n_jobs=-1, random_state = 100)


    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 200)
    search = GridSearchCV(estimator = rfc, param_grid=tuned_params,
                          cv = skf.split(X, y), scoring='neg_log_loss')
    search.fit(X, y)

    best_rfc[col_name] = search.best_estimator_
    best_params[col_name] = search.best_params_
    
    joblib.dump(best_rfc[col_name], f'./TrainedModels/RandForest/rf_{i}.joblib')
    current += 1
    print("--- Completed %s, %s, %.4f mins ---" % (current, col_name, (time.process_time() - start_time) / 60))
    
with ThreadPoolExecutor(max_workers=4) as t:
    futures = []
    for i, (col_name, y) in enumerate(y_train.iteritems()):
        futures.append(t.submit(search_model, i, col_name, y))

    for future in as_completed(futures):
        print(future.result())
        
# for i, (col_name, y) in tqdm(enumerate(y_train.iteritems())):
#     if i in [22, 34, 69, 81, 82, 121, 137, 141, 172]:
#         search_model(i, col_name, y);
too_skew = [22, 34, 69, 81, 82, 121, 137, 141, 172]

Start 1 
Start 2 
Start 3 
Start 4 
--- Completed 1, 5-alpha_reductase_inhibitor, 13.7320 mins ---
Start 5 
None
--- Completed 2, 11-beta-hsd1_inhibitor, 14.3008 mins ---
Start 6 
None
--- Completed 3, acat_inhibitor, 15.8760 mins ---
Start 7 
None
--- Completed 4, acetylcholine_receptor_agonist, 16.8310 mins ---
Start 8 
None
--- Completed 5, acetylcholinesterase_inhibitor, 15.4062 mins ---
Start 9 None

--- Completed 6, acetylcholine_receptor_antagonist, 16.3242 mins ---
Start 10 
None
--- Completed 7, adenosine_receptor_agonist, 16.0154 mins ---
Start 11 
None
--- Completed 8, adenosine_receptor_antagonist, 16.5768 mins ---
Start 12 
None
--- Completed 9, adenylyl_cyclase_activator, 14.5750 mins ---
Start 13 
None
--- Completed 10, adrenergic_receptor_agonist, 16.6154 mins ---
Start 14 None

--- Completed 11, adrenergic_receptor_antagonist, 16.5065 mins ---
Start 15 
None
--- Completed 12, akt_inhibitor, 16.6648 mins ---
Start 16 
None
--- Completed 13, aldehyde_dehydrogenase_inhibi

--- Completed 110, hmgcr_inhibitor, 16.7297 mins ---
Start 114 
None
--- Completed 111, hsp_inhibitor, 17.0359 mins ---
Start 115 
None
--- Completed 112, igf-1_inhibitor, 16.4969 mins ---
Start 116 
None
--- Completed 113, ikk_inhibitor, 17.1609 mins ---
Start 117 
None
--- Completed 114, imidazoline_receptor_agonist, 16.7937 mins ---
Start 118 
None
--- Completed 115, immunosuppressant, 17.2672 mins ---
Start 119 
None
--- Completed 116, insulin_secretagogue, 15.0659 mins ---
Start 120 
None
--- Completed 117, insulin_sensitizer, 15.5487 mins ---
Start 121 
None
--- Completed 118, integrin_inhibitor, 15.6081 mins ---
Start 122 
None
--- Completed 119, jak_inhibitor, 16.4607 mins ---
Start 123 None

--- Completed 120, kit_inhibitor, 16.1107 mins ---
Start 124 None

--- Completed 121, laxative, 14.5802 mins ---
Start 125 
None
--- Completed 122, leukotriene_inhibitor, 15.0229 mins ---
Start 126 
None
--- Completed 123, leukotriene_receptor_antagonist, 16.6367 mins ---
Start 127 
None
-

In [62]:
best_params

{'5-alpha_reductase_inhibitor': {'class_weight': {0: 1, 1: 971},
  'max_depth': 3,
  'max_features': 25},
 '11-beta-hsd1_inhibitor': {'class_weight': {0: 1, 1: 1214},
  'max_depth': 3,
  'max_features': 25},
 'acat_inhibitor': {'class_weight': {0: 1, 1: 728},
  'max_depth': 3,
  'max_features': 25},
 'acetylcholine_receptor_agonist': {'class_weight': {0: 1, 1: 95},
  'max_depth': 4,
  'max_features': 25},
 'acetylcholinesterase_inhibitor': {'class_weight': {0: 1, 1: 255},
  'max_depth': 3,
  'max_features': 25},
 'acetylcholine_receptor_antagonist': {'class_weight': {0: 1, 1: 59},
  'max_depth': 3,
  'max_features': 30},
 'adenosine_receptor_agonist': {'class_weight': {0: 1, 1: 310},
  'max_depth': 3,
  'max_features': 25},
 'adenosine_receptor_antagonist': {'class_weight': {0: 1, 1: 179},
  'max_depth': 3,
  'max_features': 25},
 'adenylyl_cyclase_activator': {'class_weight': {0: 1, 1: 1457},
  'max_depth': 3,
  'max_features': 25},
 'adrenergic_receptor_agonist': {'class_weight': {0:

In [19]:
# load saved models
best_rfc = {}
for i, col_name in enumerate(y_train.columns):
    try:
        best_rfc[col_name] = joblib.load(f'./TrainedModels/RandForest/logloss/rf_{i}.joblib')
    except:
        print(f'rfc_{i}.joblib does not exist.')

rfc_22.joblib does not exist.
rfc_34.joblib does not exist.
rfc_69.joblib does not exist.
rfc_81.joblib does not exist.
rfc_82.joblib does not exist.
rfc_121.joblib does not exist.
rfc_137.joblib does not exist.
rfc_141.joblib does not exist.
rfc_172.joblib does not exist.


In [29]:
# inference - training
y_train_pred = np.zeros(y_train.shape).astype('float')
y_train_pred_proba = np.zeros(y_train.shape).astype('float')
for i, col_name in tqdm(enumerate(y_train.columns)):
    rfc = best_rfc.get(col_name, None)
    if rfc!=None:
        y_train_pred[:,i] = rfc.predict(X_train)
        y_train_pred_proba[:,i] = rfc.predict_proba(X_train)[:,1]

206it [04:05,  1.19s/it]


In [30]:
# overall log_loss
print(log_loss(y_train.values.reshape(-1, 1), y_train_pred_proba.reshape(-1,1)))
# overall precision
print(precision_score(y_train.values.reshape(-1, 1), y_train_pred.reshape(-1,1)))
# overall precision
print(recall_score(y_train.values.reshape(-1, 1), y_train_pred.reshape(-1,1)))

0.05924467548877927
0.9286071965090493
0.9068743887103535


In [31]:
# inference - test
y_pred = np.zeros(y_test.shape).astype('float')
y_pred_proba = np.zeros(y_test.shape).astype('float')
for i, col_name in tqdm(enumerate(y_test.columns)):
    rfc = best_rfc.get(col_name, None)
    if rfc!=None:
        y_pred[:,i] = rfc.predict(X_test)
        y_pred_proba[:,i] = rfc.predict_proba(X_test)[:,1]

206it [00:49,  4.14it/s]


In [32]:
# overall log_loss
print(log_loss(y_test.values.reshape(-1, 1), y_pred_proba.reshape(-1,1)))
# overall precision
print(precision_score(y_test.values.reshape(-1, 1), y_pred.reshape(-1,1)))
# overall precision
print(recall_score(y_test.values.reshape(-1, 1), y_pred.reshape(-1,1)))

0.06704424027108556
0.7223001402524544
0.20355731225296442


In [39]:
# recall, f1, for each column
recall = np.zeros(y_train.shape[1])
f1 = np.zeros(y_train.shape[1])
for i in range(y_pred.shape[1]):
    recall[i] = recall_score(y_test.values[:,i], y_pred[:,i])
    f1[i] = f1_score(y_test.values[:,i], y_pred[:,i])
    
# best weight_ratio
ratio = np.zeros(y_train.shape[1])
# weight_ratio = np.ones(y_train.shape[1])*0.72
for i, (col_name, y) in enumerate(y_train.iteritems()):
    ratio[i] = round(len(y)/sum(y)-1)
#     rfc = best_rfc.get(col_name, None)
#     if rfc != None:
#         param = rfc.params_
#         weight_ratio[i] = round(param['class_weight'][1]/ratio[i], 2)

In [40]:
f1[f1>0.8]

array([0.84931507, 0.90322581, 0.87452471, 0.94262295, 0.88135593])

In [38]:
np.where(f1>0.9)

(array([110, 163], dtype=int64),)

In [ ]:
plt.figure(figsize=(15, 6))
# plt.plot(ratio)
plt.plot(recall)

In [ ]:
weight_ratio[recall==0]

In [ ]:
ratio[recall==0]

### Submission

In [29]:
df_submit = pd.read_csv(data_dir+'test_features.csv', index_col='sig_id')
df_submit['cp_type'].replace({'trt_cp':1., 'ctl_vehicle':0.}, inplace=True)
df_submit['cp_dose'].replace({'D1':1., 'D2':0.}, inplace=True)
df_submit.head()

# inference
y_submit_pred_proba = np.zeros((df_submit.shape[0], y_test.shape[1])).astype('float')
for i, col_name in tqdm(enumerate(y_train.columns)):
    rfc = best_rfc.get(col_name, None)
    if rfc!=None:
        y_submit_pred_proba[:,i] = rfc.predict_proba(df_submit)[:,1]
        
y_submit_proba = pd.DataFrame(y_submit_pred_proba)
y_submit_proba.index = df_submit.index
y_submit_proba.columns = y_train.columns
y_submit_proba.head()

y_submit_proba.to_csv('submission.csv')

206it [10:32,  3.07s/it]
